In [36]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, ReLU
from tensorflow.keras.models import Model

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def build_generator(latent_dim):
    input = Input(shape=(latent_dim,))
    x = Dense(8 * 8 * 128)(input)
    x = Reshape((8, 8, 128))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2DTranspose(32, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    output = Conv2DTranspose(1, kernel_size=4, strides=2, padding='same', activation='sigmoid')(x)
    return Model(input, output)

def build_discriminator(img_shape):
    input = Input(shape=img_shape)
    x = Conv2D(32, kernel_size=4, strides=2, padding='same')(input)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(64, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    output = Dense(1, activation='sigmoid')(x)
    return Model(input, output)

##Define loss functions and optimaziers

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

##Data load

In [ ]:
img_shape = (64, 64, 1)
latent_dim = 100

generator = build_generator(latent_dim)
discriminator = build_discriminator(img_shape)

transform = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

dataloader = transform.flow_from_directory(
    'data',
    target_size=(64, 64),
    color_mode='grayscale',
    batch_size=128,
    class_mode=None
)

##GAN Training

In [ ]:
num_epochs = 100
batch_size = 128

for epoch in range(num_epochs):
    for real_imgs in dataloader:
        # Actualiza el discriminador
        real_labels = tf.ones((real_imgs.shape[0], 1))
        fake_imgs = generator(tf.random.normal((real_imgs.shape[0], latent_dim)))
        fake_labels = tf.zeros((real_imgs.shape[0], 1))

        with tf.GradientTape() as tape_real, tf.GradientTape() as tape_fake:
            real_preds = discriminator(real_imgs)
            fake_preds = discriminator(fake_imgs)
            real_loss = tf.keras.losses.binary_crossentropy(real_labels, real_preds)
            fake_loss = tf.keras.losses.binary_crossentropy(fake_labels, fake_preds)
            total_loss = (real_loss + fake_loss) / 2

        real_grads = tape_real.gradient(real_loss, discriminator.trainable_weights)
        fake_grads = tape_fake.gradient(fake_loss, discriminator.trainable_weights)
        discriminator_optimizer.apply_gradients(zip(real_grads + fake_grads, discriminator.trainable_weights))

        # Actualiza el generador
        noise = tf.random.normal((batch_size, latent_dim))
        real_labels = tf.ones((batch_size, 1))

        with tf.GradientTape() as tape:
            fake_imgs = generator(noise)
            fake_preds = discriminator(fake_imgs)
            gen_loss = tf.keras.losses.binary_crossentropy(real_labels, fake_preds)

        gen_grads = tape.gradient(gen_loss, generator.trainable_weights)
        generator_optimizer.apply_gradients(zip(gen_grads, generator.trainable_weights))

    print(f"Epoch {epoch+1}/{num_epochs} | D Loss: {total_loss.numpy().mean()} | G Loss: {gen_loss.numpy().mean()}")